#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/
* https://kidger.site/thoughts/jaxtyping/
* https://github.com/patrick-kidger/torchtyping/tree/master

## Задачи для совместного разбора

In [1]:
# !pip install torchtyping

In [2]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

Scalar = TensorType[()]
patch_typeguard()

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [3]:
class Neuron:
    def __init__(self, n_features: int, bias: float) -> None:
        self.weights: TensorType["n_features"] = th.rand(n_features)
        self.bias: float = bias

    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> Scalar:
        return inputs @ self.weights + self.bias

In [4]:
import torch as th
inputs = th.tensor([1.0, 2.0, 3.0, 4.0])

In [5]:
neuron = Neuron(n_features=4, bias=0.0)
neuron.forward(inputs)

tensor(7.2591)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [6]:
class ReLU:
    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> TensorType["n_features"]:
        return th.clip(inputs, min=0)

In [7]:
inputs = th.tensor([1.0, -2.0, 3.0, -4.0])

act = ReLU()
act.forward(inputs)

tensor([1., 0., 3., 0.])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [8]:
class MSELoss:
    @typechecked
    def forward(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"]
    ) -> Scalar:
        return ((y_pred - y_true)**2).mean()

In [9]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

In [10]:
criterion = MSELoss()
loss = criterion.forward(y_pred, y_true)

In [11]:
loss

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1"></p>

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

- [x] Проверено на семинаре

In [12]:
from typing import Any

In [13]:
class Linear:
    def __init__(self, n_neurons: int, n_features: int, bias: bool = False) -> None:
        self.n_neurons = n_neurons
        self.n_features = n_features

        fgen = th.randn if bias else th.zeros
        self.weights: TensorType["n_neurons", "n_features"] = th.randn((n_neurons, n_features))
        self.bias_: TensorType["n_neurons"] = fgen((self.n_neurons,))
    
    @typechecked()
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "n_neurons"]:
        return inputs @ self.weights.T + self.bias_
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [14]:
linear = Linear(2, 4, bias=False)
linear(th.randn((3, 4)))

tensor([[ 1.6902,  3.5703],
        [-2.7186, -5.1443],
        [ 0.8565,  5.2196]])

In [15]:
linear = Linear(2, 4, bias=True)
linear(th.randn((3, 4)))

tensor([[ 1.2446, -0.7698],
        [ 0.3301,  1.5197],
        [-1.0819, -0.6548]])

<p class="task" id="2"></p>

2\. Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

- [x] Проверено на семинаре

In [16]:
linear1 = Linear(10, 4, bias=False)
linear2 = Linear(7, linear1.n_neurons, bias=False)

In [17]:
out1 = linear1(th.randn((3, 4)))
out2 = linear2(out1)
out2

tensor([[ -6.8516,  -4.2377,  -2.4726,   1.3834,   2.1468,   5.0604,   2.5530],
        [  2.4894,  -3.3481,  -5.9025,  -8.6236,   5.4970,   6.2789,  -4.1225],
        [ -6.5667, -10.3591, -11.4141,  -3.1198,   6.3144,  13.2361,   3.4592]])

In [18]:
out1.shape

torch.Size([3, 10])

In [19]:
out2.shape

torch.Size([3, 7])

### Создание функций активации

<p class="task" id="3"></p>

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

- [x] Проверено на семинаре

In [20]:
class Softmax:
    def __init__(self) -> None:
        pass

    @typechecked()
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        exp: TensorType["batch", "feats"] = th.exp(inputs)
        return exp / exp.sum(dim=1, keepdim=True)
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [21]:
inputs = th.randn((4, 3))
inputs

tensor([[-0.6354, -1.6755, -0.5476],
        [-0.1229,  0.7205,  0.5816],
        [ 0.3562, -1.0307, -0.8030],
        [-0.7964, -0.0671, -1.0595]])

In [22]:
softmax = Softmax()
softmax(inputs)

tensor([[0.4090, 0.1445, 0.4465],
        [0.1870, 0.4347, 0.3783],
        [0.6395, 0.1598, 0.2007],
        [0.2603, 0.5397, 0.2000]])

<p class="task" id="4"></p>

4 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

- [x] Проверено на семинаре

In [23]:
class ELU:
    def __init__(self, alpha: float) -> None:
        self.alpha = alpha

    @typechecked()
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        return th.where(inputs >= 0, inputs, self.alpha*(th.exp(inputs) - 1))
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [24]:
inputs = th.randn((4, 3))
inputs

tensor([[-1.3538,  0.7569, -1.9543],
        [-0.2662,  0.0815, -0.6458],
        [ 0.5719,  0.5026, -0.0398],
        [-0.1769,  0.0889, -0.9406]])

In [25]:
elu = ELU(0.001)
elu(inputs)

tensor([[-7.4173e-04,  7.5689e-01, -8.5834e-04],
        [-2.3372e-04,  8.1546e-02, -4.7576e-04],
        [ 5.7192e-01,  5.0258e-01, -3.9018e-05],
        [-1.6215e-04,  8.8903e-02, -6.0961e-04]])

### Создание функции потерь

<p class="task" id="5"></p>

5 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 2 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

- [x] Проверено на семинаре

In [26]:
class CrossEntropyLoss:
    def __init__(self) -> None:
        pass

    @typechecked()
    def forward(self, y_pred: TensorType["batch", "classes", float], y_true: TensorType["batch", int]):
        return -(th.log(y_pred).gather(1, y_true.unsqueeze(1))).mean()
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [27]:
layer = Linear(2, 4, bias=True)
out = layer.forward(th.randn(10, 4))
out

tensor([[ 0.1173, -0.2565],
        [-2.1778,  1.0960],
        [-0.0525,  1.2763],
        [-2.4723, -0.5185],
        [-1.8344, -0.3038],
        [ 0.2163, -2.3586],
        [-1.1714, -0.6689],
        [-1.7266,  1.5077],
        [-0.7502,  0.1831],
        [-1.0424,  0.1986]])

In [28]:
softmax = Softmax()
y_pred = softmax.forward(out)
y_pred

tensor([[0.5924, 0.4076],
        [0.0365, 0.9635],
        [0.2094, 0.7906],
        [0.1241, 0.8759],
        [0.1779, 0.8221],
        [0.9292, 0.0708],
        [0.3770, 0.6230],
        [0.0379, 0.9621],
        [0.2823, 0.7177],
        [0.2243, 0.7757]])

In [29]:
loss = CrossEntropyLoss()
loss.forward(y_pred, y_true=th.randint(0, 2, (10,)))

tensor(0.9579)

<p class="task" id="6"></p>

6 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

- [x] Проверено на семинаре

In [30]:
from typing import Any


class MSERegularized:
    def __init__(self, lambda_: float) -> None:
        self.lambda_ = lambda_

    @typechecked
    def data_loss(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"],
    ) -> Scalar:
        return ((y_true - y_pred) ** 2).mean()

    @typechecked
    def reg_loss(self, weights: TensorType["batch", 1]) -> Scalar:
        return th.sum(weights ** 2)

    @typechecked()
    def forward(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"],
        weights: TensorType["batch", 1],
    ) -> Scalar:
        return self.data_loss(y_pred, y_true) + self.lambda_ * self.reg_loss(weights)

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [35]:
y_pred = th.randn(2)
y_true = th.randn(2)
weights = th.randn(2).unsqueeze(1)
y_true, y_pred, th.sum(weights**2)

(tensor([ 0.6926, -0.1046]), tensor([-0.7481, -1.4347]), tensor(0.1146))

In [36]:
loss_fn = MSERegularized(lambda_=0.001)
loss_fn(y_pred, y_true, weights)

tensor(1.9225)